# Example Evaluation Code

This notebook will be very __similar__ to the code I use to evaluate your results - it is provided for __your convenience__ so that you can use it to evaluate your preprocessing results at any time before your __final submission__.

Please note that the results here will __NOT__ be the same as my evaluation results.

Let's start with loading the required packages.

In [80]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

Next you should load __your__ data. In this case, I am using a sample dataset (`GroupX.csv`) which contains 6 predictors (`X1 - X6`) and two target variables (`Y1, Y2`).

Please make sure you change the data to your __OWN__ dataset when using this code.

__NOTE__:
1. Your dataset maybe very different from the sample dataset.
2. Please follow this structure when submitting your dataset.

In [81]:
data = pd.read_csv('check_this_mean_z.csv', header=0)
data.head()

,Unnamed: 0,ticker,company_name,offer_price,price_range_higher_bound,price_range_lower_bound,first_day_trading_price,days,top_tier_dummy,positive_eps_dummy,...,number_of_sentences,number_of_words,number_of_real_words,number_of_long_sentences,number_of_long_words,number_of_positive_words,number_of_negative_words,number_of_uncertain_words,pre_IPO_price_revision,post_IPO_initial_return
0,0,AATI,ADVANCED ANALOGIC TECHNOLOGIES INC,-0.768715,-1.694966,-1.372495,-0.610555,0.113573,1.0,0.632109,...,0.119517,0.092856,0.135932,0.147526,0.225477,-0.082986,0.322172,0.031242,0,1
1,1,ABPI,ACCENTIA BIOPHARMACEUTICALS INC,-1.221783,-1.540432,-1.525793,-1.322783,0.777728,0.0,-1.780972,...,2.391737,2.229510,2.196810,2.259385,2.112897,0.244809,2.438487,2.106398,1,0
2,2,ACAD,ACADIA PHARMACEUTICALS INC,-1.448316,-0.304161,-0.299408,-1.407572,-0.577580,1.0,-1.367922,...,-1.784617,-1.663236,-1.603938,-1.600568,-1.376635,-0.119407,-1.942561,-1.641587,1,0
3,3,ACHN,ACHILLION PHARMACEUTICALS INC,-0.428915,0.313975,0.313784,-0.530391,1.992645,1.0,-1.009221,...,-0.885640,-0.957455,-0.887702,-1.105443,-0.568995,0.572603,-1.077859,-0.582834,1,1
4,4,ACLI,AMERICAN COMMERCIAL LINES INC.,1.723153,1.859314,1.846766,0.801568,-0.793565,1.0,0.056013,...,0.841531,0.589988,0.550932,0.501187,0.357158,0.099122,0.942775,0.624144,0,1


Checking your data types and make sure it follows the data dictionary would be an important step, you can do that using the `.dtypes` attribute.

__NOTE__: all __continuous__ faetures will be in `float64` data type, and all __categorical__ features will be in `int64` data type (given you already coded (per __suggest task \#6__ in the competition document) them).

In [82]:
data.dtypes

Unnamed: 0                      int64
ticker                         object
company_name                   object
offer_price                   float64
price_range_higher_bound      float64
price_range_lower_bound       float64
first_day_trading_price       float64
days                          float64
top_tier_dummy                float64
positive_eps_dummy            float64
prior_nasdaq_15day_returns    float64
share_overhang                float64
up_revision                   float64
sales                         float64
number_of_sentences           float64
number_of_words               float64
number_of_real_words          float64
number_of_long_sentences      float64
number_of_long_words          float64
number_of_positive_words      float64
number_of_negative_words      float64
number_of_uncertain_words     float64
pre_IPO_price_revision          int64
post_IPO_initial_return         int64
dtype: object

Now you need to specify your targets and predictors. __NOTE__ we have two targets here (`Y1, Y2`).

In [83]:
y1 = data.pre_IPO_price_revision
y2 = data.post_IPO_initial_return

Check the shape of the data.

In [84]:
data.shape

(682, 24)

It is very possible that you will use different sets of the predictors for `Y1` and `Y2`. Now let's define them.

First, let's define predictors for `Y1` - which will be the first 5 features in `data`.

In [85]:
cols = list(data.columns)
# first 5 features 
cols[:-3]

['Unnamed: 0',
 'ticker',
 'company_name',
 'offer_price',
 'price_range_higher_bound',
 'price_range_lower_bound',
 'first_day_trading_price',
 'days',
 'top_tier_dummy',
 'positive_eps_dummy',
 'prior_nasdaq_15day_returns',
 'share_overhang',
 'up_revision',
 'sales',
 'number_of_sentences',
 'number_of_words',
 'number_of_real_words',
 'number_of_long_sentences',
 'number_of_long_words',
 'number_of_positive_words',
 'number_of_negative_words']

Use below code to select the first 5 features as predictors for `Y1`.

In [90]:
predictors_y1 = data[cols[:-3]].drop(['Unnamed: 0','ticker','company_name','offer_price','first_day_trading_price','top_tier_dummy'],1)
predictors_y1.head()

,price_range_higher_bound,price_range_lower_bound,days,positive_eps_dummy,prior_nasdaq_15day_returns,share_overhang,up_revision,sales,number_of_sentences,number_of_words,number_of_real_words,number_of_long_sentences,number_of_long_words,number_of_positive_words,number_of_negative_words
0,-1.694966,-1.372495,0.113573,0.632109,0.677876,0.533016,0.609751,-0.678709,0.119517,0.092856,0.135932,0.147526,0.225477,-0.082986,0.322172
1,-1.540432,-1.525793,0.777728,-1.780972,-0.708853,-0.065718,-1.721321,-0.995844,2.391737,2.229510,2.196810,2.259385,2.112897,0.244809,2.438487
2,-0.304161,-0.299408,-0.577580,-1.367922,0.404646,-0.861292,-0.783092,-1.420517,-1.784617,-1.663236,-1.603938,-1.600568,-1.376635,-0.119407,-1.942561
3,0.313975,0.313784,1.992645,-1.009221,0.382032,-1.027894,-0.939830,-1.380223,-0.885640,-0.957455,-0.887702,-1.105443,-0.568995,0.572603,-1.077859
4,1.859314,1.846766,-0.793565,0.056013,-1.413007,0.036749,0.085937,1.360730,0.841531,0.589988,0.550932,0.501187,0.357158,0.099122,0.942775


Upon investigation of the data, we know we have __six__ features (`X1 - X6`) predicting `Y2`. Use similar code (as below) to select them.

In [91]:
predictors_y2 = data[cols[:-2]].drop(['Unnamed: 0','ticker','company_name','first_day_trading_price','top_tier_dummy'],1)
predictors_y2.head()

,offer_price,price_range_higher_bound,price_range_lower_bound,days,positive_eps_dummy,prior_nasdaq_15day_returns,share_overhang,up_revision,sales,number_of_sentences,number_of_words,number_of_real_words,number_of_long_sentences,number_of_long_words,number_of_positive_words,number_of_negative_words,number_of_uncertain_words
0,-0.768715,-1.694966,-1.372495,0.113573,0.632109,0.677876,0.533016,0.609751,-0.678709,0.119517,0.092856,0.135932,0.147526,0.225477,-0.082986,0.322172,0.031242
1,-1.221783,-1.540432,-1.525793,0.777728,-1.780972,-0.708853,-0.065718,-1.721321,-0.995844,2.391737,2.229510,2.196810,2.259385,2.112897,0.244809,2.438487,2.106398
2,-1.448316,-0.304161,-0.299408,-0.577580,-1.367922,0.404646,-0.861292,-0.783092,-1.420517,-1.784617,-1.663236,-1.603938,-1.600568,-1.376635,-0.119407,-1.942561,-1.641587
3,-0.428915,0.313975,0.313784,1.992645,-1.009221,0.382032,-1.027894,-0.939830,-1.380223,-0.885640,-0.957455,-0.887702,-1.105443,-0.568995,0.572603,-1.077859,-0.582834
4,1.723153,1.859314,1.846766,-0.793565,0.056013,-1.413007,0.036749,0.085937,1.360730,0.841531,0.589988,0.550932,0.501187,0.357158,0.099122,0.942775,0.624144


Below is the key part of this notebook - which generates a `logistic regression` model to predict `Y1`/`Y2`.

The code works this way:

1. We generate two lists `f1_score_lst` and `auc_lst` to store f1_score and AUC from each of the `10` runs of the model;
2. Define model:
    1. We define a `LogisticRegression()` model;
    
    2. We split predictors (`predictors_y1`) and target `y1` to training (80%) and testing (20%);
    
    3. We fit the model `clf` to the training data, then use it to predict on the testing data;
    
    4. We also defined a `10-fold cross validation` to make sure our model do not overfit - see [here](https://scikit-learn.org/stable/modules/cross_validation.html) for more info;
    
    5. We append the f1_score and AUC of current model to the lists (`f1_score_lst` and `auc_lst`) we defined earlier.
  
3. Print out average f1_score and AUC for all 10 runs;
4. Print out average average accuracy from cross validation
5. Print out confusion matrix and classification report for the __last__ model.

__NOTE__: Step 3 provides the evaluation results we need; step 4 - 5 can be used to verify the results from step 3.

In [92]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf = LogisticRegression()
    X1_train, X1_test, y1_train, y1_test = train_test_split(predictors_y1, y1, test_size=0.2, random_state=123)
    clf.fit(X1_train, y1_train)

    y1_pred = clf.predict(X1_test)

    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf, X1_train, y1_train, cv=kfold, scoring=scoring)

    

    
    #calculate f1-score and AUC
    
    clf_roc_auc = roc_auc_score(y1_test, y1_pred)
    f1_score_lst.append(precision_recall_fscore_support(y1_test, y1_pred, average='weighted')[2])
    auc_lst.append(clf_roc_auc)


print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

#result=logit_model.fit()
confusion_matrix_y1 = confusion_matrix(y1_test, y1_pred)


#print(result.summary())
print('Accuracy of classifier on test set: {:.2f}'.format(clf.score(X1_test, y1_test)))

print("10-fold cross validation average accuracy of classifier: %.3f" % (results.mean()))

print('Confusion Matrix for Logistic Regression Classfier:')
print(confusion_matrix_y1)

print('Classification Report for Logistic Regression Classfier:')
print(classification_report(y1_test, y1_pred))


F1 0.6418; AUC 0.6403 
Accuracy of classifier on test set: 0.64
10-fold cross validation average accuracy of classifier: 0.569
Confusion Matrix for Logistic Regression Classfier:
[[39 26]
 [23 49]]
Classification Report for Logistic Regression Classfier:
             precision    recall  f1-score   support

          0       0.63      0.60      0.61        65
          1       0.65      0.68      0.67        72

avg / total       0.64      0.64      0.64       137



Below code are used to evaluate model toward `Y2`. It is very similar to the code above - key difference is that `Y2` is imbalanced - so I wrote some code (under `# Begin oversampling`) to deal with that.

In [93]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf1 = LogisticRegression()
    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['post_IPO_initial_return'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('post_IPO_initial_return'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['post_IPO_initial_return'])
    del X2_train['post_IPO_initial_return']
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))


F1 0.6098; AUC 0.5743 
Accuracy of classifier on test set: 0.599
10-fold cross validation average accuracy of clf1: 0.522
Confusion Matrix for Classfier:
[[24 18]
 [37 58]]
Classification Report for Classfier:
             precision    recall  f1-score   support

          0       0.39      0.57      0.47        42
          1       0.76      0.61      0.68        95

avg / total       0.65      0.60      0.61       137

